In [1]:
package 'Structure'{
    import ScalarValues::* ; 
    import Interfaces::* ; //Looks like I need to import this one aswell.. weird ?
    
    package 'Interfaces'{ 
        port def CommunicationInterface{
            value inout data : Real; //flow properties => inout
            //no references ?
        }
        
        interface def CommunicationLink { //defining CommunicationLink INTERFACE which is an association block whose ends are ports
            end groundPort : CommunicationInterface;
            end spacePort : CommunicationInterface;
            stream spacePort::data to groundPort::data;
        }
    }
    
    
    
    block 'Mission'{ //Mission is made of 2 parts, communicating via CommunicationLink
        
        part groundStation : 'Ground Station' {
            port groundCommIF redefines CommIF;
        }
        
        part spacecraftVehicle : 'Spacecraft Vehicle'{
            port spaceCommIF redefines CommIF1;
        }
        
        interface : CommunicationLink connect            //creating interface
            groundPort => groundStation::groundCommIF to 
            spacePort  => spacecraftVehicle::spaceCommIF; 
    }
    
    
    block Data;
    block Ping;
    
    
    block 'Ground Station'{
        port inout CommIF : CommunicationInterface;      
        
        activity PingSpacecraft(); 
        activity ReceiveData(in data:Data);
        
        
        action pingSpacecraft : PingSpacecraft(in ping:Ping){
            //send signal on CommunicationLink interface 
            // or is the whole point of all this to just call SpaceCraft::TransmitData() from here ?
            send ping:Ping of null to "Spacecraft Vehicle";
        }
        
        action receiveData : ReceiveData(){ //(in data flow from 
                                            //'Spacecraft Vehicle'::transmitData::data)
            // 0. (merge node ?)
            // 1. waiting for a trigger : Data Packet arriving on CommunicationLink
            // 2. process data (flow dataframe from port to processing action)
                // (flow trigger::dataPacket to process::dataPacket;)
            // 3. repeat (recursive ? or flow to merge node)
            
            //TRY:
            first start;
            
            then[0..1] action merge : Activities::Merge; //why [0..1] ? slide 31

            then accept trigger (data:Data);
            
            then action processData(in data flow from trigger::data);
            
            then[0..1] merge;
        }
    }
    
    
    
    block 'Spacecraft Vehicle'{
        port inout CommIF1 : CommunicationInterface;
       
        activity ConsumePower();
        activity RechargeBatteries();
        activity SendDataFrames(out data:Data);
        activity TransmitData();
        
        
        
        action consumePower : ConsumePower() {
            // 0. in case trigger:DeviceOff => "stop action"
            // 1. merge node
            // 2. battery--;
            // 3a. case battery <40 : send signal 'Battery Low' to TransmitData process
            // 3b. case battery >40 : flow to merge node 1.
        }
        
        
        action rechargeBatteries:RechargeBatteries(){
            // 1. merge node
            // 2a. case battery >=100 : send signal 'Battery fully charged' then stop action
            // 2b. case battery <100 : battery++; then flow to merge node 1.
        }
        
        
        action TransmitData(){
            // 00 trigger "battery low" : go to 6.Merge
            // 1. run small Javascript Expression
            // 2. Fork (consume power + the following)
            // 3. Merge
            // 4. Decision
            // 5. Case data>0 : sendDataFrames() then go back to 3.Merge
            // 5. Case data=0 : Javascript expression
            // 6. Merge 
            // 7. Send signal Device Off to activity ConsumePower
            // 8. Send signal Transmission Done            
        }
        
        
        
        action sendDataFrames:SendDataFrames(){
            // 1. data = data-1024
            // 2. send DataPacket on CommunicationInterface
        }
        
                
        value data : Real = 102400;
        value battery : Real = 100;
        value transmissionInProgress : Boolean=false;
        value t0 : Real = 0;
        value tfinal : Real = 0;
    }

}

Package Structure (424b17ed)


In [ ]:
%publish Structure